## Please input your directory for the top level folder
folder name : SUBMISSION MODEL

In [1]:
dir_ = 'E:/Seminararbeit/Code/A1/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'

In [3]:
####################################################################################
####################### 1-2. recursive model by store & cat ########################
####################################################################################

In [4]:
ver, KKK = 'priv', 0

In [5]:
STORES = ['TX_3', 'WI_1', 'WI_2', 'WI_3'] #'CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 
CATS = ['HOBBIES','HOUSEHOLD', 'FOODS']

In [6]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [7]:
########################### Helpers
#################################################################################
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [8]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store, dept):
    
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    

    df = df[(df['store_id']==store) & (df['cat_id']==dept)]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
        
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3
    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    display(df.head())
    print(features)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES:
        for state_id in CATS:
            temp_df = pd.read_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'.pkl')
            temp_df['store_id'] = store_id
            temp_df['cat_id'] = state_id
            base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [9]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': 1,
                    'n_jobs': 1
                } 

In [10]:
########################### Vars
#################################################################################
VER = 1                        
SEED = 42                      
seed_everything(SEED)
lgb_params['seed'] = SEED
N_CORES = psutil.cpu_count()


#LIMITS and const
TARGET      = 'sales'
START_TRAIN = 700 
END_TRAIN   = 1941 - 28*KKK
P_HORIZON   = 28        
USE_AUX     = False

remove_features = ['id','cat_id', 'state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_store_id_dept_id_mean','enc_store_id_dept_id_std',
                   'enc_item_id_store_id_mean','enc_item_id_store_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'processedgrid_part_1.pkl'
PRICE    = processed_data_dir+'processedgrid_part_2.pkl'
CALENDAR = processed_data_dir+'processedgrid_part_3.pkl'
LAGS     = processed_data_dir+'processedlags_df_28.pkl'
MEAN_ENC = processed_data_dir+'processedmean_encoding_df.pkl'


SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [11]:
########################### Train Models
#################################################################################
for store_id in STORES:
    for state_id in CATS:
        print('Train', store_id, state_id)

        grid_df, features_columns = get_data_by_store(store_id, state_id)

        train_mask = grid_df['d']<=END_TRAIN
        valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
        preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)

        train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                           label=grid_df[train_mask][TARGET])

        valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                           label=grid_df[valid_mask][TARGET])

        grid_df = grid_df[preds_mask].reset_index(drop=True)
        keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
        grid_df = grid_df[keep_cols]

        d_sales = grid_df[['d','sales']]
        substitute = d_sales['sales'].values
        substitute[(d_sales['d'] > END_TRAIN)] = np.nan
        grid_df['sales'] = substitute

        grid_df.to_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'.pkl')
        del grid_df, d_sales, substitute

        seed_everything(SEED)
        estimator = lgb.train(lgb_params,
                              train_data,
                              valid_sets = [valid_data],
                              callbacks=[lgb.log_evaluation(1, True)]
                              
                              )
        
        display(pd.DataFrame({'name':estimator.feature_name(),
                              'imp':estimator.feature_importance()}).sort_values('imp',ascending=False).head(25))
        
        model_name = model_dir+'lgb_model_'+store_id+'_'+state_id+'_v'+str(VER)+'.bin'
        pickle.dump(estimator, open(model_name, 'wb'))

        
        del train_data, valid_data, estimator
        gc.collect()

        MODEL_FEATURES = features_columns

Train TX_3 HOBBIES


,id,d,sales,item_id,dept_id,release,sell_price,price_max,price_min,price_std,...,rolling_mean_tmp_1_30,rolling_mean_tmp_1_60,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_30,rolling_mean_tmp_7_60,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_30,rolling_mean_tmp_14_60
0,HOBBIES_1_002_TX_3_evaluation,700,0.0,HOBBIES_1_002,HOBBIES_1,5,3.970703,3.970703,3.970703,0.000000,...,0.199951,0.183350,0.285645,0.357178,0.199951,0.183350,0.428467,0.285645,0.233276,0.166626
1,HOBBIES_1_004_TX_3_evaluation,700,2.0,HOBBIES_1_004,HOBBIES_1,7,3.769531,4.640625,3.769531,0.420898,...,2.500000,2.349609,4.144531,3.142578,2.367188,2.400391,2.142578,2.000000,2.066406,2.199219
2,HOBBIES_1_005_TX_3_evaluation,700,0.0,HOBBIES_1_005,HOBBIES_1,16,2.980469,2.980469,1.500000,0.188965,...,0.399902,0.466553,1.142578,0.643066,0.533203,0.649902,0.142822,0.214233,0.433350,0.616699
3,HOBBIES_1_006_TX_3_evaluation,700,1.0,HOBBIES_1_006,HOBBIES_1,109,1.099609,1.099609,0.959961,0.059204,...,0.799805,0.600098,1.000000,0.785645,0.733398,0.600098,0.571289,0.856934,0.600098,0.549805
4,HOBBIES_1_007_TX_3_evaluation,700,0.0,HOBBIES_1_007,HOBBIES_1,126,7.878906,7.941406,7.878906,0.017059,...,0.166626,0.099976,0.142822,0.071411,0.099976,0.066650,0.000000,0.142822,0.066650,0.083313


['item_id', 'dept_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_store_id_dept_id_mean', 'enc_store_id_dept_id_std', 'enc_item_id_store_id_mean', 'enc_item_id_store_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60

,name,imp
0,item_id,101618
22,tm_w,22173
21,tm_d,21915
55,rolling_mean_180,20923
56,rolling_std_180,20536
54,rolling_std_60,19695
14,event_name_1,19542
52,rolling_std_30,18617
50,rolling_std_14,18285
48,rolling_std_7,17835


Train TX_3 HOUSEHOLD


...

,name,imp
0,item_id,119367
14,event_name_1,29943
22,tm_w,25025
21,tm_d,24578
55,rolling_mean_180,20809
56,rolling_std_180,20733
54,rolling_std_60,18453
52,rolling_std_30,16390
53,rolling_mean_60,16156
57,rolling_mean_tmp_1_7,15632


Train WI_3 FOODS


,id,d,sales,item_id,dept_id,release,sell_price,price_max,price_min,price_std,...,rolling_mean_tmp_1_30,rolling_mean_tmp_1_60,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_30,rolling_mean_tmp_7_60,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_30,rolling_mean_tmp_14_60
0,FOODS_1_001_WI_3_evaluation,700,0.0,FOODS_1_001,FOODS_1,0,2.240234,2.240234,1.750000,0.112671,...,0.300049,0.516602,0.142822,0.142822,0.500000,0.483398,0.142822,0.428467,0.533203,0.549805
1,FOODS_1_002_WI_3_evaluation,700,0.0,FOODS_1_002,FOODS_1,0,8.882812,9.476562,7.878906,0.491211,...,0.399902,0.399902,0.285645,0.500000,0.433350,0.383301,0.714355,0.428467,0.399902,0.416748
2,FOODS_1_003_WI_3_evaluation,700,0.0,FOODS_1_003,FOODS_1,0,2.880859,3.230469,2.880859,0.156860,...,0.366699,0.583496,0.285645,0.357178,0.566895,0.633301,0.428467,0.500000,0.533203,0.616699
3,FOODS_1_004_WI_3_evaluation,700,7.0,FOODS_1_004,FOODS_1,105,1.780273,1.959961,1.500000,0.103088,...,8.664062,10.953125,7.714844,7.285156,9.570312,12.070312,6.855469,9.210938,11.429688,13.000000
4,FOODS_1_005_WI_3_evaluation,700,4.0,FOODS_1_005,FOODS_1,3,3.279297,3.539062,2.939453,0.175537,...,1.900391,1.666992,2.142578,2.427734,1.766602,1.849609,2.714844,2.500000,1.700195,1.750000


['item_id', 'dept_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_store_id_dept_id_mean', 'enc_store_id_dept_id_std', 'enc_item_id_store_id_mean', 'enc_item_id_store_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60

,name,imp
0,item_id,134238
14,event_name_1,33121
21,tm_d,20154
22,tm_w,20105
55,rolling_mean_180,16398
57,rolling_mean_tmp_1_7,15876
61,rolling_mean_tmp_7_7,14831
56,rolling_std_180,14546
48,rolling_std_7,14290
54,rolling_std_60,13344
